In [12]:
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import data_helpers
from hanziconv import HanziConv

In [13]:
weibo_m_o = pd.read_csv('./data/Weibo_M.csv')
weibo_o = pd.read_csv('./data/Weibo.csv')
wechat_o = pd.read_csv('./data/Wechat.csv')
news_forum_o = pd.read_csv('./data/News_forum.csv')

In [14]:
weibo_m = weibo_m_o[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
weibo = weibo_o[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
wechat = wechat_o[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
news_forum = news_forum_o[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')

In [15]:
weibo_m.columns = ['CONTENT','AREAS AFFECTED']
weibo.columns = ['CONTENT','AREAS AFFECTED']

In [23]:
data = pd.concat([weibo_m,weibo,wechat,news_forum])

In [24]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('life', 'Life')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('sponsored', 'Sponsored')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('news', 'News')
data = data.reset_index(drop=True)

In [25]:
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')

In [26]:
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('News', 'Brands')
for i in range(len(data)):
    if ((data['AREAS AFFECTED'][i] != 'Customer Service')):
        data['AREAS AFFECTED'][i] = 'others'

In [27]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1)

In [29]:
trainpath = './data/processed/training/'
if not os.path.exists(trainpath):
    os.makedirs(trainpath)
for the_file in os.listdir(trainpath):
    file_path = os.path.join(trainpath, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
testpath = './data/processed/testing/'
if not os.path.exists(testpath):
    os.makedirs(testpath)
for the_file in os.listdir(testpath):
    file_path = os.path.join(testpath, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [30]:
for x in data['AREAS AFFECTED'].unique():
    np.savetxt(trainpath+ x.replace('/',' ') +'.utf8',\
                   train[train['AREAS AFFECTED'] == x]['CONTENT'].values, fmt='%s')
    np.savetxt(testpath+ x.replace('/',' ') +'.utf8',\
                   test[test['AREAS AFFECTED'] == x]['CONTENT'].values, fmt='%s')

In [31]:
data['AREAS AFFECTED'].value_counts()

others              29142
Customer Service     1752
Name: AREAS AFFECTED, dtype: int64

# Sentiment data

In [76]:
weibo_m_s = weibo_m_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
weibo_s = weibo_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
wechat_s = wechat_o[['CONTENT','SENTIMENT']].dropna(axis=0, how = 'any')

In [77]:
weibo_m_s.columns = ['CONTENT','SENTIMENT']
weibo_s.columns = ['CONTENT','SENTIMENT']

In [78]:
data = pd.concat([weibo_m_s,weibo_s,wechat_s])

In [79]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data = data.reset_index(drop=True)

In [80]:
length = []
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')
    length.append(len(data['CONTENT'][i]))
length = np.array(length)

In [81]:
data = data[length <= 1000]

In [82]:
mypath = './data/processed/'
if not os.path.exists(mypath):
    os.makedirs(mypath)
for the_file in os.listdir(mypath):
    file_path = os.path.join(mypath, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [83]:
for x in data['SENTIMENT'].unique():
    np.savetxt(mypath+ x.replace('/',' ') +'.utf8',\
                   data[data['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')

In [84]:
data['SENTIMENT'].value_counts()

Neutral       12972
Positive        447
Irrelevant      253
Negative        250
Name: SENTIMENT, dtype: int64

In [85]:
data['SENTIMENT'].value_counts()

Neutral       12972
Positive        447
Irrelevant      253
Negative        250
Name: SENTIMENT, dtype: int64

In [86]:
from gensim.models import Word2Vec
w2vModel = Word2Vec.load('runs/trained_word2vec.model.512')

In [87]:
len(w2vModel.wv.vocab)

3577

In [89]:
from gensim.models import KeyedVectors
w2vModel = KeyedVectors.load_word2vec_format('runs/wiki.zh.vec')

In [90]:
vocab_obj = w2vModel.wv.vocab
word = u'河北'
print word
print vocab_obj[word].index

河北
3038


In [94]:
np.random.permutation(np.arange(100))

array([80, 37, 63,  4,  9, 95, 97, 78, 86, 40, 75, 46, 13, 64, 50,  7, 93,
       65, 55, 26, 83, 53, 32, 99, 39,  2, 31, 60, 52, 28, 23, 11, 81, 73,
       48, 15, 33,  1, 19, 30, 56, 27,  6, 36, 61, 98, 42, 58, 82, 18, 49,
       38, 12, 90, 35, 62, 17, 43, 91, 71,  8, 89, 79, 20, 14, 44, 41,  3,
       84, 96, 21, 51, 68, 69, 47, 88, 24, 70, 45, 29, 25, 34, 66, 87, 22,
       74,  0, 16, 67, 57, 94, 54, 59,  5, 92, 77, 85, 10, 72, 76])

In [34]:
np.sum(np.array([1,2]) == np.array([1,3]))

1